# Restricted Two Body Problem: Elliptical Orbits Around a Central Mass

This is the general case of a Keperian orbit.<br>
A light body (e.g. a planet) orbits a heavy central body (e.g. the sun).  The orbit is an ellipse with the primary at one focus.